In [2]:
import nltk
import string
import numpy as np
import operator
import math
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer


def normalize_line(line):
    words = [stemmer.stem(word) for word in tokenizer.tokenize(line) if word not in stopwords.words('english')]
    return words


documents = {}
N = 0
with open('cran.all.1400', 'r') as f:
    tokenizer = RegexpTokenizer(r'\w+')
    stemmer = SnowballStemmer("english")

    line = f.readline()
    while line:
        document = {}
        document_id = line.split()[1]
    
        f.readline()  # skip .T line

        line = f.readline()
        title_words = []
        while not line.startswith('.A'):
            title_words.extend(normalize_line(line))
            line = f.readline()
        document['title'] = title_words

        line = f.readline()
        while not line.startswith('.W'):
            line = f.readline()

        line = f.readline()
        annotation_words = []
        while not line.startswith('.I') and line:
            annotation_words.extend(normalize_line(line))
            line = f.readline()
        document['annotation'] = annotation_words

        documents[document_id] = document
        N += 1   
    
L_title = 0  # average length of annotation in documents
for document_index, document in documents.iteritems():
    L_title += len(document['annotation'])
    L_title /= float(N)


L_annotation = 0  # average length of annotation in documents
for document_index, document in documents.iteritems():
    L_annotation += len(document['annotation'])
    L_annotation /= float(N)


In [6]:
def create_inverted_index(document_field_name):
    invert_index = {}
    for document_index, document in documents.iteritems():
        for word in document[document_field_name]:
            if word in invert_index.keys() and document_index in invert_index[word].keys():
                pass
            else:
                if word not in invert_index:
                    invert_index[word] = {document_index: document[document_field_name].count(word)}
                else:
                    invert_index[word][document_index] = document[document_field_name].count(word)

    return invert_index

In [43]:
def search_in_index(index, query_words, field_name, b, k1, rsv_function_type, k2):
    founded_documents_indexes = set()
    for query_word in query_words:
        if query_word in index:
            founded_documents_indexes.update(index[query_word].keys())

    documents_with_rsv = {}
    for founded_document in founded_documents_indexes:
        document_rsv = rsv(query_words, documents[founded_document][field_name], index, field_name, b, k1, rsv_function_type)
        documents_with_rsv[founded_document] = document_rsv

    return documents_with_rsv


def rsv(query_words, document_words, inverse_index, field_name, b, k1, rsv_function_type):
    if field_name == 'title':
        L = L_title
    else:
        L = L_annotation

    rsv = 0
    for query_word in query_words:
        if query_word in inverse_index:
            Nt = len(inverse_index[query_word])
            ftd = document_words.count(query_word)
            Ld = len(document_words)
            if rsv_function_type == 1:
                rsv += math.log1p(1 + (N - Nt + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld * L + ftd))
            elif rsv_function_type == 2:
                rsv += math.log1p(1 + (N + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld * L + ftd))
            elif rsv_function_type == 3:
                 rsv += math.log1p(1 + (N + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld * L + ftd))
    return rsv


In [18]:
title_index = create_inverted_index('title')
annotation_index = create_inverted_index('annotation')

In [44]:
def answer_for_field(field_name, b=0.75, k1=1.2, rsv_function_type=1):
    with open('answer', 'w') as result_file:
        with open('cran.qry', 'r') as f:
            line = f.readline()
            query_number = 0
            while line:
                line = f.readline()  # skip .W line 

                query_words = []
                while not line.startswith('.I') and line:
                    query_words.extend(normalize_line(line))
                    line = f.readline()

                documents_with_rsv = search_in_index(title_index, query_words, field_name, b, k1, rsv_function_type)
                top10 = [elem[0] for elem in
                         sorted(documents_with_rsv.items(), key=operator.itemgetter(1), reverse=True)[:10]]

                query_number += 1
                for top in top10:
                    result_file.write('%s %s\n' % (query_number, top))


In [45]:
answer_for_field('title')
%run ./eval.py


mean precision: 0.239555555556
mean recall: 0.351973187993
mean F-measure: 0.285082114808
MAP@10: 0.282763867193


In [46]:
answer_for_field('annotation')
%run ./eval.py

mean precision: 0.276888888889
mean recall: 0.409783052588
mean F-measure: 0.330476220923
MAP@10: 0.339283966014


Поиск по индексу, построенному по аннатациям получился более точным. Так как аннотации больше по размеру и там больше информации.

In [33]:
def eval():
    groundtruth_file = 'qrel_clean'
    answer_file = 'answer'
    
    q2reld = {} 
    for line in open(groundtruth_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2reld.keys():
            q2reld[qid] = set()
        q2reld[qid].add(did)        
    
    q2retrd = {}
    for line in open(answer_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2retrd.keys():
            q2retrd[qid] = []
        q2retrd[qid].append(did)               
    
    N = len(q2retrd.keys())
    precision = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2retrd[q]) for q in q2retrd.keys()]) / N
    recall = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2reld[q]) for q in q2retrd.keys()]) / N
    print("mean precision: {}\nmean recall: {}\nmean F-measure: {}"\
          .format(precision, recall, 2*precision*recall/(precision+recall)))

    # MAP@10
    import numpy as np
    
    MAP = 0.0
    for q in q2retrd.keys():
        n_results = min(10, len(q2retrd[q]))
        avep = np.zeros(n_results)
        for i in range(n_results):
            avep[i:] += q2retrd[q][i] in q2reld[q]
            avep[i] *= (q2retrd[q][i] in q2reld[q]) / (i+1.0)
        MAP += sum(avep) / min(n_results, len(q2reld[q]))
    print("MAP@10: {}".format(MAP/N))
    return MAP/N


best_result = 0
for k in np.arange(1.2, 2.1, 0.1):
    for b in np.arange(0, 1.1, 0.1):
        answer_for_field('annotation', b, k)
        eval_result = eval()
        if eval_result > best_result:
            best_result = eval_result
            best_params = (k, b)
            
print best_params

mean precision: 0.250222222222
mean recall: 0.373345427043
mean F-measure: 0.299628508699
MAP@10: 0.285003286582


mean precision: 0.257333333333
mean recall: 0.382614154697
mean F-measure: 0.307710797058
MAP@10: 0.303300482209


mean precision: 0.266222222222
mean recall: 0.392916869389
mean F-measure: 0.317393410431
MAP@10: 0.318745466952


mean precision: 0.268
mean recall: 0.395306029611
mean F-measure: 0.319436312068
MAP@10: 0.325556248425


mean precision: 0.274666666667
mean recall: 0.405992939663
mean F-measure: 0.327660776078
MAP@10: 0.331720813387


...

mean precision: 0.281333333333
mean recall: 0.414585189003
mean F-measure: 0.335201979626
MAP@10: 0.341968552112


mean precision: 0.279555555556
mean recall: 0.411635061158
mean F-measure: 0.332975782442
MAP@10: 0.339820703508


mean precision: 0.277777777778
mean recall: 0.410280835138
mean F-measure: 0.331270901955
MAP@10: 0.340199386915


mean precision: 0.276888888889
mean recall: 0.409186700597
mean F-measure: 0.330282122299
MAP@10: 0.339124598975


mean precision: 0.275111111111
mean recall: 0.405723497134
mean F-measure: 0.327888860962
MAP@10: 0.335661776686
(2.0000000000000009, 0.60000000000000009)


In [37]:
print best_result

answer_for_field('annotation', 0.6, 2)
eval_result = eval()

0.341968552112


mean precision: 0.281333333333
mean recall: 0.414585189003
mean F-measure: 0.335201979626
MAP@10: 0.341968552112


7. Попробуйте заменить функцию вычисления IDF-составляющей в формуле BM25 выше на один из других вариантов, представленных на лекции

Для этого убрал Nt из числителя под лагорифмом (так было сказано на лекции)

In [ ]:
на сколько важно учитывать длину жокумента